In [14]:
from tool import *
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
# import matplotlib.pyplot as plt

## 1. 지난 주 대비 금주 SP 변동폭 정리

In [15]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

In [16]:
ver_table = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-30:, ]

,Planweek,From Site,Category,Created_at,SP
744,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401291025,409153.0
745,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401291546,412325.0
746,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401300855,412581.0
747,2024-01-29(W05),OS_TCL_CN_P,Latest26w,202401310947,408789.0
777,2024-02-05(W06),OS_TCL_CN_P,Final26w,202402081246,430204.0
778,2024-02-05(W06),OS_TCL_CN_P,Final52w,202402081307,539755.0
779,2024-02-05(W06),OS_TCL_CN_P,Latest26w,202402050920,415119.0
780,2024-02-05(W06),OS_TCL_CN_P,Latest26w,202402051504,431920.0
781,2024-02-05(W06),OS_TCL_CN_P,Latest26w,202402060857,426394.0
812,2024-02-12(W07),OS_TCL_CN_P,Final26w,202402131354,472946.0


In [17]:
i1 = 936 # 이번주
i2 = 914 # 지난주

p_wk = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Planweek']
vendor = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['From Site'].split('_')[1]
ver = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Category']

sp1 = spdb.merge(spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i1], :4])
sp2 = spdb.merge(spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i2], :4])
sp2 = sp2[sp2['Week Name'] >= sp1['Week Name'].min()]

sp = pd.concat([sp1, sp2])
sp['Month'] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

sp = sp[sp['Week Name'] >= [get_weekname_from(p_wk, i) for i in range(-3, 1)][0]] # 이번주 기준 3주전부터 Sorting
sp = sp[sp['Month'] <= '2024-05'] # 24년 4월까지만 한정
sp.insert(7, 'Series', sp['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
sp.insert(7, 'Inch', sp['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left')
sp.loc[sp['Division']=='PC', 'SCM'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='PC', 'MKT'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'SCM'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'MKT'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
us_b2b = sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
sp = sp.fillna('N/A')

In [18]:
gap = sp.groupby(['Region', 'Country', 'MKT', 'SCM', 'Planweek', 'Created_at', 'To Site', 'Category', 'Series', 'Month', 'Week Name'])['SP'].sum().unstack(['Planweek', 'Category', 'Created_at']).fillna(0).astype(int).sort_index(axis=1)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap = gap.unstack('Month').fillna(0).astype(int)
gap['Sum of Gap'] = gap['Gap'].sum(axis=1)
gap = gap.reset_index()

In [19]:
countries = gap['Country'].unique()

In [20]:
# 국가별 탭으로 엑셀 파일 만들기
df_all = []
for country in countries:
    df = gap[gap['Country'] == country].copy()
    df = df.reset_index(drop=True).set_index(['Region', 'Country', 'MKT', 'SCM', 'To Site', 'Series', 'Week Name'])
    df_all.append(df)

updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
filepath = f'D:/SP검토/{updated_time}_{p_wk}_TCL SP Review_by_country.xlsx'
with pd.ExcelWriter(filepath) as writer:
    for i, df in enumerate(df_all):
        df.to_excel(writer, sheet_name=f'{countries[i]}', startrow=1)

## 변동 내역을 Tracking할 SP version을 선택

In [21]:
spdb = pd.merge(spdb, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [22]:
spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index()[-50:]

,Planweek,Category,Created_at,SP
103,2024-01-08(W02),Final52w,202401100925,1247317.0
104,2024-01-08(W02),Latest26w,202401081020,959810.0
105,2024-01-08(W02),Latest26w,202401081508,950599.0
106,2024-01-08(W02),Latest26w,202401091016,950297.0
107,2024-01-08(W02),Latest26w,202401101010,949748.0
108,2024-01-08(W02),Latest26w,202401110911,949478.0
109,2024-01-15(W03),Final26w,202401161405,1021663.0
110,2024-01-15(W03),Final52w,202401161425,1332142.0
111,2024-01-15(W03),Latest26w,202401151014,1023539.0
112,2024-01-15(W03),Latest26w,202401151513,1023219.0


In [23]:
idxs = [99, 102, 109, 115, 121, 127, 132, 137, 142, 146, 152] # index 넘버를 선택
df = pd.DataFrame()
for i in idxs:
    df = pd.concat([df, spdb.merge(spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :-1])]).reset_index(drop=True)    

In [24]:
vendor = 'OS_TCL_CN_P'
search_period = ['2024-03', '2024-04', '2024-05']
# search_period = [get_weekname_from(get_weekname(datetime.datetime(2024, 1, 29)), i) for i in range(0, 6)]
df = df[df['From Site'].str.contains(vendor)]

In [25]:
df['Month'] = df['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')
df.insert(7, 'Series', df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
df.insert(7, 'Inch', df['Mapping Model.Suffix'].apply(lambda x:x[:2]))
df = df[df['Month'].isin(search_period)]
df.loc[df['Mapping Model.Suffix'].str.contains('27MC41D'), 'SP'] = df.loc[df['Mapping Model.Suffix'].str.contains('27MC41D'), 'SP'] * 2 # 캐나다 Costco 2 pack 모델을 개별 모니터 개수 기준으로 변환하기 위함

In [26]:
wb = load_workbook(filepath)

for sheet_name in wb.sheetnames:
    confirm_wk = '2024-04-08(W15)'
    color = PatternFill(fgColor='9999FF', fill_type='solid') # violet
    new_confirm_color = PatternFill(fgColor='32CD32', fill_type='solid') # LimeGreen
    confirmed_color = PatternFill(fgColor='006400', fill_type='solid') # DarkGreen
    border_style = Side(border_style="thin", color="000000")
    border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
    sh = wb[sheet_name]
    sh['A1'] = '■ Color Description '
    sh['A2'].fill = confirmed_color
    sh['A3'].fill = new_confirm_color
    sh['B2'] = 'Confired SP'
    sh['B3'] = 'New confirmed SP'
    sh['G1'] = 'Sub TTL'
    for i in range(8, sh.max_column+1):
        col_letter = get_column_letter(i)
        sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
        sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=2, min_col=7, max_row=5, max_col=sh.max_column):
        for cell in row:
            cell.fill = color
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    merge_range = f'{get_column_letter(sh.max_column)}2:{get_column_letter(sh.max_column)}5'
    sh.merge_cells(merge_range)
    for cell in sh[merge_range][0]:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    for row in sh.iter_rows(min_row=6, min_col=1, max_row=6, max_col=7):
        for cell in row:
            cell.fill = color
    for row in sh.iter_rows(min_row=7, min_col=7, max_row=sh.max_row, max_col=sh.max_column):
        for cell in row:
            cell.border = border
            cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=7, min_col=7):
        if row[0].value < confirm_wk:
            for cell in row:
                cell.fill = confirmed_color
        elif row[0].value == confirm_wk:
            for cell in row:
                cell.fill = new_confirm_color
    sh.freeze_panes = 'H7'
    sh.column_dimensions['A'].width = 8
    sh.column_dimensions['B'].width = 10
    sh.column_dimensions['G'].width = 17.5
    sh.sheet_view.showGridLines = False
    for i in range(8, sh.max_column+1):
        sh.column_dimensions[get_column_letter(i)].width = 6.5
    
    df_bar = df.groupby(['Country', 'Planweek', 'Series', 'Month']).sum('SP').loc[sheet_name].reset_index()
    df_bar['Month'] = df_bar['Month'].astype('category').cat.set_categories(search_period, ordered=True)
    fig = px.bar(df_bar, x='Planweek', y='SP', color='Month', facet_col='Series', title=f'{sheet_name} Final SP Variation',
                 category_orders={'Month':search_period, 'Planweek':sorted(df['Planweek'].unique())}, width=1200, text_auto=True)
#     try:
#         df_bar = df.groupby(['Country', 'Planweek', 'Series', 'Week Name']).sum('SP').loc[sheet_name].reset_index()
#     except:
#         continue
#     fig = px.bar(df_bar, x='Planweek', y='SP', color='Week Name', facet_col='Series', title=f'{sheet_name} Final SP Variation',
#                  category_orders={'Week Name':search_period, 'Planweek':sorted(df['Planweek'].unique())}, width=1200, text_auto=True)
    
    image_filepath = f'D:/figure/{sheet_name}_weekly_sp_variation.png'
    fig.write_image(image_filepath)
    
    plot_image = Image(image_filepath)
    sh.add_image(plot_image, f'A{sh.max_row+2}')

# Summary sheet 작성
wb.create_sheet('Summary')
ws = wb['Summary']
fig = px.bar(df.groupby(['Planweek', 'Inch', 'Month']).sum().reset_index(), x='Planweek', y='SP', color='Month', facet_col='Inch', width=1200)
fig.write_image('D:/figure/TCL_weekly_sp_variation_by_inch.png')
ws.add_image(Image('D:/figure/TCL_weekly_sp_variation_by_inch.png'), 'A2')

fig = px.bar(df.groupby(['Planweek', 'Region', 'Month']).sum().reset_index(), x='Planweek', y='SP', color='Month', facet_col='Region', facet_col_wrap=4, height=1000, width=1200, text='SP')
fig.write_image('D:/figure/TCL_weekly_sp_variation_by_region.png')
ws.add_image(Image('D:/figure/TCL_weekly_sp_variation_by_region.png'), 'A27')

wb.save(filepath)
wb.close()

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_1340\3588755229.py:71: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_1340\3588755229.py:75: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

